In [51]:
pip install eli5

     ---------------------------------------- 0.0/216.2 kB ? eta -:--:--
     ------------------- ------------------ 112.6/216.2 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 216.2/216.2 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/47.1 kB ? eta -:--:--
   ---------------------------------------- 47.1/47.1 kB ? eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107762 sha256=4a819653dc872b7be8c101084ecd404227f929b14173bea66e20ad81dabe026b
  Stored in directory: c:\users\uffeb\

In [53]:
pip install imblearn

   ---------------------------------------- 0.0/258.0 kB ? eta -:--:--
   ----------------- ---------------------- 112.6/258.0 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 258.0/258.0 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [58]:
import pandas as pd
import openpyxl as openpyxl
import random 
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek

In [ ]:
import os
os.getcwd()

In [ ]:
data.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
#file_path = r"C:\Users\uffeb\OneDrive - Aarhus universitet\Kandidat\Master\Speciale\Speciale---Stibo\Data\WA_Fn-UseC_-Telco-Customer-Churn.csv"
file_path = r"C:\Users\josep\OneDrive - Aarhus universitet\Speciale\Speciale---Stibo\Data\WA_Fn-UseC_-Telco-Customer-Churn.csv"

In [13]:
## Loading the ADULT dateset from UCI repo 
#Avaliable at https://archive.ics.uci.edu/dataset/2/adult

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features # Importing all features 
y = adult.data.targets # Importing target varialbes

In [14]:
y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})


C:\Users\uffeb\AppData\Local\Temp\ipykernel_35588\1846786675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})


In [15]:
y.value_counts()

income
<=50K     37155
>50K      11687
Name: count, dtype: int64

In [24]:
data =  pd.concat([X, y], axis=1)

In [17]:
# Checking the data types of all the columns
data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

In [25]:
## Converting the Predictor Variable into Numeric 
data['income']=data['income'].map({'<=50K':0,'>50K':1})
data['income'].value_counts()

income
0    37155
1    11687
Name: count, dtype: int64

In [45]:
data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,9,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,7,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [44]:
data=pd.get_dummies(data, drop_first = True)

In [48]:
from sklearn.model_selection import train_test_split

y = data['income'].values
features = [col for col in data.columns if col not in ['income']]
X = data[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3, stratify=y)

(34189, 100) (34189,)
(14653, 100) (14653,)


In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

param_grid = {
    'max_depth': [2],
    'max_features': ['sqrt'],
    'min_samples_leaf': [10],
    'min_samples_split': [5],
    'n_estimators': [10]
    }

# Initialize the RandomForestClassifier (you can choose any other classifier)
classifier = RandomForestClassifier(random_state=0)

# Train the classifier on the training set
n_estimators = 100
grid_search_rf = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_params_rf = grid_search_rf.best_params_
print("Best Hyperparameters:", best_params_rf)


# Use the best model for predictions
best_model_rf = grid_search_rf.best_estimator_
y_pred_rf = best_model_rf.predict(X_test)

Best Hyperparameters: {'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 5, 'n_estimators': 10}


In [65]:
import eli5
from eli5.sklearn import PermutationImportance

imp = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(imp, feature_names = X_test.columns.tolist())

ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (c:\Users\uffeb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\metaestimators.py)

In [47]:
# Set the random seed
random.seed(2137)
np.random.seed(2137)

y = data['income']
X = data.drop(['income'], axis=1)

# Get the numeric and categorical column names
objecttypes_num = X.select_dtypes(include=['float64', 'int64']).columns
objecttypes_cat = X.select_dtypes(include=['object']).columns

# Transformer for numerical columns
transformer_num = Pipeline([
    ('scaler', StandardScaler())
])

# Transformer for categorical columns
transformer_cat = Pipeline([
    ('encoder', OneHotEncoder(drop='first'))
])

# Preprocessor for all features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_num, objecttypes_num),
        ('cat', transformer_cat, objecttypes_cat)
    ], remainder='passthrough'
)

# Standardize and one-hot encode the features
X_processed = preprocessor.fit_transform(X)


# Get the column names after transformation
processed_colnames = preprocessor.get_feature_names_out()
processed_colnames = [name.replace('num__', '').replace('cat__', '') for name in processed_colnames]

# Convert the processed features to a DataFrame
##_processed = pd.DataFrame(X_processed, columns=processed_colnames)


In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_processed.drop('Churn', axis=1), df_processed['Churn'], test_size=0.2, random_state=42)

param_grid = {
    'max_depth': [2],
    'max_features': ['sqrt'],
    'min_samples_leaf': [10],
    'min_samples_split': [5],
    'n_estimators': [10]
    }

# Initialize the RandomForestClassifier (you can choose any other classifier)
classifier = RandomForestClassifier(random_state=0)

# Train the classifier on the training set
n_estimators = 100
grid_search_rf = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_params_rf = grid_search_rf.best_params_
print("Best Hyperparameters:", best_params_rf)


# Use the best model for predictions
best_model_rf = grid_search_rf.best_estimator_
y_pred_rf = best_model_rf.predict(X_test)


In [ ]:
y_pred_rf[0:50]

In [ ]:
X_test.head()

## Model explainability - Local and Global interpretations

In [ ]:
pip install shap

In [ ]:
pip install matplotlib

In [ ]:
y_test

In [ ]:
best_model_rf

In [ ]:
import shap
import matplotlib.pyplot as plt

# Create an explainer for the best model
explainer_rf = shap.Explainer(best_model_rf)

#X_test_col = X_test.columns

# Calculate SHAP values for a single prediction (change the index as needed)
#shap_values_rf = explainer_rf(X_test.iloc[1])

# Calculate SHAP values for a set of predictions
shap_values_rf = explainer_rf(X_test)

# Create an output for Churn = 1, for all X observations in the test set
shap.summary_plot(shap_values_rf[:, :, 1], X_test)

In [ ]:
# Viser fordelingen af variabler for prediction i række 1, fra perspektivet churn = 0 (begge inputs kan ændres)
shap.plots.waterfall(shap_values_rf[1, :, 1])

In [ ]:
shap_values_rf[:, :, 1].shape

In [ ]:
# Showing a dataframe of each - 0.26 is the base value for churn = 1, for all observations. Each variable then has a SHAP-value +- the BV
shap_df = pd.DataFrame(
    np.c_[shap_values_rf[:, :, 1].base_values, shap_values_rf[:, :, 1].values],
    columns = ["ChurnBaseValue"] + list(X_test.columns)
)

In [ ]:
shap_df.to_csv('shap_df_testset.csv', index=False)  # Set index=False to not write row numbers as the first column

## Next steps
1. lav agg average calc
2. afprøv flere viz
3. kan vi allerede nu lave LLM ovenpå disse SHAP værdier?